In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup

In [3]:
df = pd.read_json('data/train_new.json')

In [4]:
df.head().T

0  \
acct_type                                             fraudster_event   
approx_payout_date                                         1266062400   
body_length                                                      3852   
channels                                                            5   
country                                                            US   
currency                                                          USD   
delivery_method                                                     0   
description         <p><a href="http://s432.photobucket.com/albums...   
email_domain                                                gmail.com   
event_created                                              1262739706   
event_end                                                  1265630400   
event_published                                           1.26311e+09   
event_start                                                1265594400   
fb_published                                                        0   
gts                                                                 0   
has_analytics                                                       0   
has_header                                                          1   
has_logo                                                            0   
listed                                                              y   
name                99 HOUR "NO SLEEP" SUPER BOWL CELEBRITY WEEKEN...   
name_length                                                        60   
num_order                                                           0   
num_payouts                                                         0   
object_id                                                      527017   
org_desc                                                                
org_facebook                                                        0   
org_name                        Party Starz Ent & Diverse Int'l Group   
org_twitter                                                         0   
payee_name                                                              
payout_type                                                             
previous_payouts                                                   []   
sale_duration                                                      29   
sale_duration2                                                     33   
show_map                                                            1   
ticket_types        [{u'event_id': 527017, u'cost': 25.0, u'availa...   
user_age                                                           36   
user_created                                               1259613950   
user_type                                                           1   
venue_address                                   717 Washington Avenue   
venue_country                                                      US   
venue_latitude                                                25.7775   
venue_longitude                                              -80.1334   
venue_name                                INK Nightclub - South Beach   
venue_state                                                        FL   

                                                                    1  \
acct_type                                                     premium   
approx_payout_date                                         1296720000   
body_length                                                      3499   
channels                                                            0   
country                                                            US   
currency                                                          USD   
delivery_method                                                     1   
description         <p>Join us for a quick, one-night, community-b...   
email_domain                                                  ruf.org   
event_created                                       

In [3]:
header = df.columns

### Everett

In [4]:
#Replace boolean strings with 1,0
df[df.columns[15:30]] = df[df.columns[15:30]].replace(['y','n', 'CHECK', 'ACH'],[1,0,1,0])

for c in ('org_facebook', 'org_twitter', 'has_header'):
    df[c] = df[c].astype('category')

### Crystal

In [5]:
#change to categorical and rename
df['channels'] = df['channels'].apply(lambda x: "Channel_" + str(x))

#only 3 values + missing so changed into categorical
df['delivery_method'] = df['delivery_method'].fillna("Missing")
df['delivery_method'] = df['delivery_method'].apply(lambda x: "DeliveryType_" + str(x))

num_people_domain = df['email_domain'].value_counts(dropna=False)
df['email_domain_num_people'] = df['email_domain'].apply(lambda x: num_people_domain[x])
df['email_domain_morethan20'] = df['email_domain'].apply(lambda x: True if x>=20 else False)

iso_co = pd.read_csv("data/iso_countries.csv")
iso_co.index = iso_co['iso 3166 country']
del iso_co['iso 3166 country']
iso_co = iso_co.fillna("NorthA")
iso_co.loc['']="Missing"
iso_co.loc['NA'] = 'AF'

df['continent'] = df['country'].apply(lambda x: iso_co.loc[x][0])

In [6]:
# TF/IDF
vectorizer1 = TfidfVectorizer(encoding='english',
                            stop_words='english',
                            strip_accents="ascii",
                          # token_pattern=r'\w{3,}',
                           max_features=100)

text_vec = df['description'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
text_vec1 = zip(text_vec, df['name'], df['org_name'])
text_vec1 = [ ''.join(ln) for ln in text_vec1]

In [7]:
r = vectorizer1.fit_transform(text_vec1)
columns = vectorizer1.get_feature_names()
pd.DataFrame(r.toarray(),columns=columns).head()

00   10   11   12   15        20      2011  2012     2013   30  \
0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000   0.0  0.00000  0.0   
1  0.000000  0.0  0.0  0.0  0.0  0.118757  0.132081   0.0  0.00000  0.0   
2  0.110951  0.0  0.0  0.0  0.0  0.000000  0.000000   0.0  0.00000  0.0   
3  0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000   0.0  0.13997  0.0   
4  0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000   0.0  0.00000  0.0   

     ...          use      want  way  wine      work  workshop     world  \
0    ...     0.000000  0.000000  0.0   0.0  0.000000       0.0  0.000000   
1    ...     0.000000  0.124529  0.0   0.0  0.000000       0.0  0.000000   
2    ...     0.130813  0.000000  0.0   0.0  0.000000       0.0  0.000000   
3    ...     0.000000  0.079146  0.0   0.0  0.000000       0.0  0.149206   
4    ...     0.000000  0.000000  0.0   0.0  0.163398       0.0  0.165953   

        www      year     years  
0  0.000000  0.000000  0.000000  
1  0.000000  0.000000  0.000000  
2  0.000000  0.000000  0.000000  
3  0.065455  0.215691  0.000000  
4  0.000000  0.000000  0.160099  

[5 rows x 100 columns]

### Miles

In [8]:
# Helper functions
def previous_payouts_add_ts(pp_list):
    """Add timestamp field to previous_payouts"""
    for d in pp_list:
        if d.get('created'):
            d['ts'] = pd.Timestamp(d.get('created'))
    return pp_list

def previous_payouts_first_date(x):
    """Return earliest payout date"""
    date_list = [pd.Timestamp(row.get('created')) for row in x if row.get('created')]
    if len(date_list):
        return min(date_list)
    else:
        return None

def amount_fn(x, fn, colname='amount'):
    """Return fn of amount if we have amounts, else None"""
    amount_list = [row.get(colname, 0) for row in x]
    if len(amount_list):
        return fn(amount_list)
    else:
        return 0

In [9]:
# Helper functions
def previous_payouts_add_ts(pp_list):
    """Add timestamp field to previous_payouts"""
    for d in pp_list:
        if d.get('created'):
            d['ts'] = pd.Timestamp(d.get('created'))
    return pp_list

def previous_payouts_first_date(x):
    """Return earliest payout date"""
    date_list = [pd.Timestamp(row.get('created')) for row in x if row.get('created')]
    if len(date_list):
        return min(date_list)
    else:
        return None

def amount_fn(x, fn, colname='amount'):
    """Return fn of amount if we have amounts, else None"""
    amount_list = [row.get(colname, 0) for row in x]
    if len(amount_list):
        return fn(amount_list)
    else:
        return 0

In [10]:

# Turn dates into timestamps
df['previous_payouts'] = df['previous_payouts'].map(previous_payouts_add_ts)

# Add count of previous payouts
df['previous_payouts_count'] = df['previous_payouts'].map(lambda x: len(x))
                                    
# Add sum of previous payouts
df['previous_payouts_sum'] = df['previous_payouts'].map(lambda x: amount_fn(x, sum))
                                    
# Add date of earliest payout
df['previous_payouts_earliest'] = df['previous_payouts'].map(previous_payouts_first_date)
                                    
# Add maximum payout amount
df['previous_payouts_max'] = df['previous_payouts'].map(lambda x: amount_fn(x, max))

# Add median payout amount
df['previous_payouts_mean'] = df['previous_payouts'].map(lambda x: amount_fn(x, np.mean))

# Add median payout amount
df['previous_payouts_median'] = df['previous_payouts'].map(lambda x: amount_fn(x, np.median))

In [11]:
df[[c for c in df.columns if 'previous_payouts' in c]].head()

previous_payouts  previous_payouts_count  \
0                                                 []                       0   
1  [{u'name': u'RUF', u'created': u'2010-10-01 01...                      49   
2  [{u'name': u'Danielle Severn', u'created': u'2...                      36   
3  [{u'name': u'', u'created': u'2010-11-09 01:10...                      21   
4  [{u'name': u'Arts and Business Council or Grea...                      49   

   previous_payouts_sum previous_payouts_earliest  previous_payouts_max  \
0                  0.00                       NaT                  0.00   
1             803323.86       2010-10-01 01:10:09              86855.80   
2              54302.40       2010-06-30 01:10:47               4020.21   
3               4384.37       2010-11-09 01:10:15                999.38   
4              12915.02       2010-12-18 01:11:29               3439.38   

   previous_payouts_mean  previous_payouts_median  
0               0.000000                     0.00  
1           16394.364490                  5425.92  
2            1508.400000                  1446.94  
3             208.779524                    75.13  
4             263.571837                   105.00

### ticket_types

In [12]:
## Helper functions
def cost_max(x):
    """Return max of x['cost']"""
    return amount_fn(x, max, colname='cost')
def cost_min(x):
    """Return min of x['cost']"""
    return amount_fn(x, min, colname='cost')
def quantity_sum(x):
    """Return sum of x['quantity_total']"""
    return amount_fn(x, sum, colname='quantity_total')
def total_value(x):
    """Return sum of x['quantity_total'] * x['cost']"""
    return sum(d.get('quantity_total', 0) * d.get('cost', 0) for d in x)

In [13]:
df['ticket_types_count'] = df['ticket_types'].map(lambda x: len(x))
df['ticket_types_cost_min'] = df['ticket_types'].map(cost_min)
df['ticket_types_cost_max'] = df['ticket_types'].map(cost_max)
df['ticket_types_quantity_sum'] = df['ticket_types'].map(quantity_sum)
df['ticket_types_total_value'] = df['ticket_types'].map(total_value)
df['ticket_types_cost_mean'] = df['ticket_types_total_value'] / df['ticket_types_quantity_sum']
    

In [14]:
df[[c for c in df.columns if 'ticket_types' in c]].head(2)

ticket_types  ticket_types_count  \
0  [{u'event_id': 527017, u'cost': 25.0, u'availa...                   3   
1  [{u'event_id': 786878, u'cost': 35.0, u'availa...                   2   

   ticket_types_cost_min  ticket_types_cost_max  ticket_types_quantity_sum  \
0                   25.0                  550.0                        920   
1                   35.0                   35.0                        100   

   ticket_types_total_value  ticket_types_cost_mean  
0                   36000.0               39.130435  
1                    3500.0               35.000000

### venue_country

In [15]:
df['user_venue_country_same'] = (df['country'] == df['venue_country']).astype(int)

### venue_address

In [16]:
df['venue_address_blank'] = (df['venue_address'].map(len) == 0).astype(int)

### user_created

In [17]:
cols = ['user_created']

for col in cols:
    df[col] = pd.to_datetime(df[col], unit='s')